# Bitcoin data cleaning


Dataset obtained from https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data, where there was over 4 million entries of data. Due to large amount of data, the size of the file was 300,000 KB, and made it too big to push to github. We used the fear and greed index dataset as a restriction, where it starts from 2018-02-01. So we will only use the data from 2018-02-01, to match the time frame. 

After cleaning the data, we then export it in to the file data

In [1]:
# Importing the libraries to be used:
from datetime import datetime
import numpy as np
import pandas as pd
import requests

In [2]:
def date_convertor(timestamp):
    return datetime.fromtimestamp(float(timestamp))

In [3]:
bitcoin_data = pd.read_csv("data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv", sep=",",parse_dates = ["Timestamp"], date_parser = date_convertor) 

In [4]:
bitcoin_clean = bitcoin_data.dropna()
bitcoin_clean = bitcoin_clean[1900000:]
print(bitcoin_clean)

                  Timestamp      Open      High       Low     Close  \
3098119 2017-11-25 02:03:00   8185.00   8185.00   8180.11   8185.00   
3098120 2017-11-25 02:04:00   8185.00   8185.00   8185.00   8185.00   
3098121 2017-11-25 02:05:00   8185.00   8185.14   8185.00   8185.14   
3098122 2017-11-25 02:06:00   8182.01   8182.01   8182.01   8182.01   
3098123 2017-11-25 02:07:00   8185.13   8185.14   8185.13   8185.14   
...                     ...       ...       ...       ...       ...   
4857372 2021-03-30 19:56:00  58714.31  58714.31  58686.00  58686.00   
4857373 2021-03-30 19:57:00  58683.97  58693.43  58683.97  58685.81   
4857374 2021-03-30 19:58:00  58693.43  58723.84  58693.43  58723.84   
4857375 2021-03-30 19:59:00  58742.18  58770.38  58742.18  58760.59   
4857376 2021-03-30 20:00:00  58767.75  58778.18  58755.97  58778.18   

         Volume_(BTC)  Volume_(Currency)  Weighted_Price  
3098119      0.520570        4259.672912     8182.705868  
3098120      0.182275        

In [5]:
timestamp = np.array(bitcoin_clean["Timestamp"])

In [13]:
bitcoin_day = []
start_index = 0
prev = timestamp[0]
for i in range(timestamp.shape[0]):
    if str(prev)[:10] not in str(timestamp[i])[:10]:
        if str(prev)[:10] >= "2018-01-31":
            bitcoin_day.append(bitcoin_clean.iloc[i])
        prev = timestamp[i]

In [14]:
bitcoin_pd = pd.DataFrame(bitcoin_day)

In [15]:
day_time = np.array(bitcoin_pd["Timestamp"])
print(day_time.shape[0])
for i in range(day_time.shape[0]):
    new_date = str(day_time[i])[5:10] + str(day_time[i])[:5]
    bitcoin_pd.replace(day_time[i], new_date)

1154


In [39]:
bitcoin_pd = bitcoin_pd.rename(columns={"Timestamp":"Date"})
bitcoin_pd = bitcoin_pd.reset_index()
print(bitcoin_pd)

        index       Date      Open      High       Low     Close  \
0     3195916 2018-02-01  10092.62  10109.93  10092.62  10109.93   
1     3197356 2018-02-02   8630.00   8630.00   8594.12   8630.00   
2     3198796 2018-02-03   8408.72   8419.79   8400.10   8414.00   
3     3200236 2018-02-04   9044.01   9044.01   9030.29   9031.93   
4     3201676 2018-02-05   8278.56   8286.74   8254.07   8254.07   
...       ...        ...       ...       ...       ...       ...   
1149  4850416 2021-03-26  52449.23  52537.81  52449.23  52527.35   
1150  4851856 2021-03-27  54821.66  54856.01  54821.66  54856.01   
1151  4853296 2021-03-28  56095.81  56158.59  56095.81  56149.01   
1152  4854736 2021-03-29  55361.25  55417.15  55361.25  55401.67   
1153  4856176 2021-03-30  57069.04  57072.89  57033.66  57051.13   

      Volume_(BTC)  Volume_(Currency)  Weighted_Price  
0         0.270443        2733.799973    10108.600991  
1         9.315129       80242.200123     8614.179884  
2         1.044

 # Extracting Fear and Greed Index

In [17]:
response = requests.get("https://api.alternative.me/fng/?limit=1600&format=csv&date_format=us")

In [25]:
fngIndex = []
startIndex = 0
endIndex = 0
count = 0
label = ["Date", "fng_value", "fng_classification"]
for i in response.iter_lines():
    line = i.decode("utf-8").strip("\t\n").split(",")
    if "03-30-2021" == line[0]:
        startIndex = count
    if "02-01-2018" == line[0]:
        endIndex = count+1
    count+=1
    fngIndex.append(line)
fngIndex = fngIndex[startIndex:endIndex]

In [26]:
fngArr = np.array(fngIndex)
fngPd = pd.DataFrame(fngArr, columns = label)
print(fngPd)

            Date fng_value fng_classification
0     03-30-2021        72              Greed
1     03-29-2021        72              Greed
2     03-28-2021        74              Greed
3     03-27-2021        65              Greed
4     03-26-2021        54            Neutral
...          ...       ...                ...
1146  02-05-2018        11       Extreme Fear
1147  02-04-2018        24       Extreme Fear
1148  02-03-2018        40               Fear
1149  02-02-2018        15       Extreme Fear
1150  02-01-2018        30               Fear

[1151 rows x 3 columns]


# Combining Fear and Greed w/ Bitcoin data

In [40]:
df3 =bitcoin_pd.append(fngPd)
print(df3)

          index                 Date      Open      High       Low     Close  \
0     3195916.0  2018-02-01 00:00:00  10092.62  10109.93  10092.62  10109.93   
1     3197356.0  2018-02-02 00:00:00   8630.00   8630.00   8594.12   8630.00   
2     3198796.0  2018-02-03 00:00:00   8408.72   8419.79   8400.10   8414.00   
3     3200236.0  2018-02-04 00:00:00   9044.01   9044.01   9030.29   9031.93   
4     3201676.0  2018-02-05 00:00:00   8278.56   8286.74   8254.07   8254.07   
...         ...                  ...       ...       ...       ...       ...   
1146        NaN           02-05-2018       NaN       NaN       NaN       NaN   
1147        NaN           02-04-2018       NaN       NaN       NaN       NaN   
1148        NaN           02-03-2018       NaN       NaN       NaN       NaN   
1149        NaN           02-02-2018       NaN       NaN       NaN       NaN   
1150        NaN           02-01-2018       NaN       NaN       NaN       NaN   

      Volume_(BTC)  Volume_(Currency)  

# Converting to CSV

In [ ]:
#if not(os.path.exists("data/fear-n-greed-02-01-2018-to-03-30-2021.csv")):
#    DataFrame(bitcoin_day).to_csv("data/bitstamp_2018-02-31_to_2021_03_30.csv") 

In [ ]:
#if not(os.path.exists("data/bitstamp_2018-02-01_to_2021_03_31.csv")):
#    DataFrame(bitcoin_day).to_csv("data/bitstamp_2020-03-31_to_2021_03_31.csv")